In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preparation

In [3]:
## input FNA set ## 88701 x 20 df
df_ROI_FNA = pd.read_csv(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonIn2\postPredict_tPRfNRfDR_NonInFNA2_all.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [4]:
df_ROI_FNA

,pixel_id,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,-0.382212,0.614860,0.093588,0.0,0.0,0.0,-0.305501,0.0,...,0,1,0.0,1.0,FP,0.951854,0.048146,0.161665,0.130747,0.869253
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,-0.382212,0.399367,-0.203629,0.0,0.0,0.0,-0.305501,0.0,...,0,1,0.0,1.0,FP,0.951854,0.048146,0.161665,0.130747,0.869253
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,-0.382212,0.386685,0.197185,0.0,0.0,0.0,-0.305501,0.0,...,0,1,0.0,1.0,FP,0.951854,0.048146,0.161665,0.130747,0.869253
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,-0.382212,0.319038,0.140703,0.0,0.0,0.0,-0.305501,0.0,...,0,1,0.0,1.0,FP,0.951854,0.048146,0.161665,0.130747,0.869253
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,0.0,-0.382212,0.391484,0.137820,0.0,0.0,0.0,-0.305501,0.0,...,0,1,0.0,1.0,FP,0.951854,0.048146,0.161665,0.130747,0.869253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24760,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,0.0,0.119769,-0.222611,-0.203629,0.0,0.0,0.0,0.144291,0.0,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.584010,1.000000,0.000000
24761,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_5...,0.0,0.144545,0.569072,0.428367,0.0,0.0,0.0,0.039793,0.0,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.584010,1.000000,0.000000
24762,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_6...,0.0,0.198568,0.440340,0.559590,0.0,0.0,0.0,0.120134,0.0,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.584010,1.000000,0.000000
24763,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,0.0,0.045366,0.548928,0.505295,0.0,0.0,0.0,0.151603,0.0,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.584010,1.000000,0.000000


In [5]:
df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

In [6]:
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [7]:
df_ROI_FNA

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3,1,84,0.0,-0.382212,0.614860,0.093588,0.0,0.0,...,0,1,0.0,1.0,FP,0.951854,0.048146,0.161665,0.130747,0.869253
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3,53,67,0.0,-0.023600,-0.222611,-0.203629,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.584010,1.000000,0.000000
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3,53,66,0.0,-0.382212,0.165008,-0.203629,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.584010,1.000000,0.000000
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3,53,65,0.0,0.002577,0.205793,-0.203629,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.584010,1.000000,0.000000
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3,53,64,0.0,-0.382212,-0.222611,-0.203629,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.584010,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24760,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_9...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,68,96,0.0,0.193888,-0.222611,0.225063,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
24761,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_9...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,68,95,0.0,0.036326,0.220769,0.146068,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
24762,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_9...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,68,94,0.0,0.195074,-0.222611,0.209807,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
24763,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,68,106,0.0,0.191491,0.166925,0.192857,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253


In [9]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [10]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI]

In [11]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI"]

In [12]:
sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI]

In [13]:
HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_6...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,10,69,0.0,0.208450,-0.222611,0.284084,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
1,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,11,70,0.0,0.051041,-0.222611,0.344125,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
2,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_6...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,69,0.0,0.138206,-0.222611,0.360091,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
3,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,77,0.0,0.077341,-0.222611,0.447207,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
4,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,78,0.0,0.135105,0.145592,0.413546,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,138,0.0,0.322806,-0.222611,0.242797,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
10298,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,139,0.0,0.254461,0.146804,0.218574,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
10299,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,140,0.0,0.330261,0.212512,0.267351,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253
10300,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,141,0.0,0.354435,-0.222611,0.276726,0.0,0.0,...,1,1,0.0,1.0,TP,0.951854,0.048146,0.161665,0.130747,0.869253


# 2. Prediction (made)

## deploy models for training set ##
model_xgb = jl.load(r"H:\3_output_raMSIn\modelXGBoost.joblib")
#
## deploy model ##
predicted0n1_train = model_xgb.predict(X_train)
trainDEFSDf["predicted0n1"] = predicted0n1_train
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\df_train24_std_0n1.csv"
trainDEFSDf.to_csv(savePath)

pTP_train = model_xgb.predict_proba(X_train)
trainDEFSDf["p(0)"] = list(pd.DataFrame(pTP_train)[0])
trainDEFSDf["p(1)"] = list(pd.DataFrame(pTP_train)[1])
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\df_train24_std_0n1_pTP.csv"
trainDEFSDf.to_csv(savePath)

In [14]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(13.074742446242135, '%')

In [15]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(95.18540089303048, '%')

# 3. EDA

In [16]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [17]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [35]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Reds")

In [36]:
def genAROItif(sample_ROI, sample_, name, mz, vmax_=1):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=1, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_NonIn2/{name}_{mz}.tif", bbox_inches = 'tight', pad_inches = 0)

In [20]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_NonIn2/raMSIn_NonInFNA_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(49530, 439)

In [21]:
raMSI_ML = df_raMSI_ML.reset_index()

In [22]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [23]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [24]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [25]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [26]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3]

In [37]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="p(1)"
    

counter = 0

for im in range(len(sample_ROI_list)):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:00:54.670598
0:00:34.456751
